<a href="https://colab.research.google.com/github/RyosukeHanaoka/TechTeacher/blob/main/Semantic_Segmentation%EF%BC%9A%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E5%89%8D%E5%87%A6%E7%90%86%E3%83%BB%E6%8B%A1%E5%BC%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Semantic Segmentation：データの前処理・拡張

###使用するライブラリのインポート

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

###データの前処理・拡張を設定

####__SegmentationDataset__
SegmentationDatasetクラスは、PyTorchのカスタムデータセットを定義するためのものです。PyTorchのDatasetを継承しており、主に3つのメソッド(__init__, __len__, __getitem__)を実装することで、データローディングや前処理の流れをカスタマイズすることができます。
####__init__
image_dir: 画像ファイルが保存されているディレクトリへのパスを受け取り、インスタンス変数として保存します。

mask_dir: マスクファイルが保存されているディレクトリへのパスを受け取り、インスタンス変数として保存します。

transform: 画像とマスクの前処理やデータ拡張を行うための関数またはクラスのインスタンスを受け取り、インスタンス変数として保存します。デフォルトはNoneで、前処理やデータ拡張を行わない場合を意味します。

self.image_list: os.listdir(image_dir)を使用して、image_dirディレクトリ内のファイル名のリストを取得し、インスタンス変数として保存します。これにより、データセット内の各画像に簡単にアクセスできるようになります。

####__len__
__len__はPythonの特殊メソッドの一つで、組み込み関数len()を対象のオブジェクトに適用したときに呼び出されるメソッドです。Datasetクラスを継承する際には、この__len__メソッドを定義する必要があります。このメソッドは、データセットの総数（つまり画像の総数）を返すように設計されています。

具体的には、self.image_listにはimage_dirに存在する画像ファイルの名前のリストが格納されています。そのため、len(self.image_list)はそのディレクトリにある画像ファイルの総数を返します。

この__len__メソッドのおかげで、後の処理でlen(dataset_object)のようにして、データセットのサイズ（画像の数）を簡単に取得することができます。

####__getitem__

os.path.join(self.image_dir, self.image_list[idx]):
これは、指定されたディレクトリ（self.image_dir）とファイル名（self.image_list[idx]）を組み合わせて、そのファイルのフルパスを作成しています。マスクについても同様の操作をしています。

Image.open(image_path).convert("RGB"):
これにより、先程作成したフルパスから画像を開き（読み込み）、RGBモードでの画像として変換します。convert("RGB")は、元の画像がRGBモードでない場合や、アルファチャンネルを持っている場合にRGBモードへ変換するために使用します。マスクはモノクロームとして読み込むため、convert("L")を使用しています。

if self.transform::
この部分は、前処理やデータ拡張を適用するためのものです。self.transformは初期化の際に指定されるtransform関数（または関数の組み合わせ）を保持しています。

この関数は、通常、torchvision.transformsモジュールから提供される関数を使用して作成されますが、カスタムの変換関数を使用することもできます。この関数は、入力として画像とマスクを受け取り、前処理やデータ拡張を行った画像とマスクを返すように設計されていることが期待されます。

この例では、self.transformがNoneでない場合（すなわち、何らかの変換関数が指定されている場合）にのみ、変換が適用されます。

この__getitem__関数は、指定されたインデックス（idx）に対応する画像とマスクのペアを返すように設計されています。これにより、データローダーがバッチ単位で画像とマスクのペアを取得する際に、この関数が連続的に呼び出されます。

In [ ]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform#「このクラスの中で後から使えるように、transformの値をインスタンス変数に保存する」という意味
        self.image_list = os.listdir(image_dir)
        #この__init__メソッドは、データセットクラスがインスタンス化されるとき（例：dataset = SegmentationDataset(image_dir, mask_dir)）に一度だけ呼び出される。

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_list[idx])
        mask_path = os.path.join(self.mask_dir, self.image_list[idx])

        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.transform:
            image, mask = self.transform(image, mask)

        return image, mask

###データセットクラスの作成 (画像とマスクのペアを返す)

####__SegmentationTransform__
SegmentationTransformクラスは、セマンティックセグメンテーションのデータ変換のためのカスタムクラスを初期化する部分です。

####__init__
output_size: この値は、出力される画像のサイズを指定します。多くのディープラーニングモデルでは、入力画像のサイズが一定であることが必要です。この変数を使用して、すべての画像が同じサイズにリサイズされることを保証します。

meanとstd: これらは、データ正規化のための平均と標準偏差を指定します。ディープラーニングモデルの学習を効果的に行うために、入力データを正規化（平均0、標準偏差1）するのが一般的です。これらの値は、データセット全体の画像の平均と標準偏差に基づいていることが多いです。ただし、一般的な値（例：Imagenetの平均と標準偏差）を使用することもよくあります。

####__call__
transforms.Resize(self.output_size):torchvision.transformsモジュールのResizeクラスをインスタンス化しています。このインスタンス化されたクラスは、引数として与えられたサイズ（self.output_size）に画像をリサイズする機能を持っています。

Horizontal Flip（水平反転）:単純なData Augmentationの手法で、50%の確率で入力画像と対応するマスクを水平方向に反転させます。このようなランダムな変換は、モデルがトレーニングデータに過剰適合するのを防ぎ、汎化性能を向上させることを目的としています。

ToTensor:これにより、PIL画像（またはnumpy配列）がPyTorchのテンソルに変換されます。この変換では、画像のピクセル値が[0, 255]の範囲から[0.0, 1.0]の範囲に正規化されます。

Normalization（正規化）:これにより、テンソル形式の画像が指定された平均値と標準偏差で正規化されます。これはネットワークの収束を速くし、トレーニングの安定性を向上させるための一般的な手法です。正規化する際の平均値と標準偏差は、通常、使用するデータセットの統計に基づいています。

この関数の最後で、正規化された画像とマスクのテンソルが返されます。これにより、モデルのトレーニングや評価に使用できるようになります。

In [ ]:
class SegmentationTransform:
    def __init__(self, output_size, mean, std):
        self.output_size = output_size
        self.mean = mean
        self.std = std

    def __call__(self, image, mask):
        # Resize：#torchvision.transformsモジュールのResizeクラスをインスタンス化している。引数として与えたサイズ（self.output_size）に画像をリサイズする。
        resize = transforms.Resize(self.output_size)#変数'resize'にリサイズ機能を持つオブジェクト'Resize'を代入している。
        image = resize(image)#実際に画像とマスクをリサイズ
        mask = resize(mask)#同上

        # Random Horizontal Flip
        if np.random.random() > 0.5:
            image = transforms.functional.hflip(image)
            mask = transforms.functional.hflip(mask)

        # ToTensor & Normalize
        image = transforms.functional.to_tensor(image)
        mask = transforms.functional.to_tensor(mask)
        image = transforms.functional.normalize(image, self.mean, self.std)

        return image, mask.squeeze(dim=0)

transform = SegmentationTransform(output_size=(384, 288), mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

mask.squeeze(dim=0)の.squeeze()メソッドは、指定された次元のサイズが1である場合に、その次元を取り除きます。

例えば、テンソルの形状が[1, 512, 512]である場合（これは通常、1チャンネルのグレースケール画像を意味します）、.squeeze(dim=0)を適用すると、形状が[512, 512]のテンソルになります。

このコンテキストでは、マスク画像が1つのチャンネルを持つグレースケール画像であることが期待されているため、.squeeze(dim=0)を使用して余分な次元を取り除いていると推測されます。

ただし、実際にこの操作が必要かどうかは、後続の処理やモデルの設計によります。多くのニューラルネットワークはバッチ形式で画像を受け取るため、画像の形状が[B, C, H, W]（Bはバッチサイズ、Cはチャンネル数、HとWはそれぞれ高さと幅）であることを期待します。この場合、余分なチャンネル次元を取り除く必要はないかもしれません。

###DataLoaderの作成

####ディレクトリの指定:

これは、あなたのGoogle Drive内にある画像とマスクの格納場所を指定しています。your_dataset_pathは実際のデータセットのパスに置き換える必要があります。

####Datasetの作成:

ここでは、先程定義したSegmentationDatasetクラスを使用して、セマンティックセグメンテーションのためのデータセットを作成しています。このデータセットは、指定された変換（transform）を使用して、入力画像と対応するマスクを適切に前処理します。ここでのtransformは、SegmentationTransformクラスのインスタンスを指すものとして想定されています。

####DataLoaderの作成:
DataLoaderは、データセットからバッチを効率的に読み込むためのツールです。batch_sizeはモデルのトレーニング中に一度に処理される画像の数を指定します。shuffle=Trueは、各エポックの開始時にデータセットをシャッフルすることを意味します。これは、モデルがトレーニングデータに過剰適合するのを防ぐための一般的な手法です。

要するに、このコードスニペットは、あなたのデータセットを前処理し、モデルのトレーニングや評価のためにバッチ単位でデータを供給する準備をしています。

In [ ]:
image_dir = "/content/drive/MyDrive/your_dataset_path/Images"
mask_dir = "/content/drive/MyDrive/your_dataset_path/Masks"
dataset = SegmentationDataset(image_dir, mask_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)